In [9]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from collections import Counter
# Defines the dataset paths
jaffe_train_path = "models/JAFFE-[70,30]/train/"
jaffe_test_path = "models/JAFFE-[70,30]/test/"
ck_train_path = "CK_dataset/train/"
ck_test_path = "CK_dataset/test/"


In [10]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
# Function to read in and detect facial images. 
def load_detect_faces(path):
    images = []
    labels = []
    for label in os.listdir(path):
        label_path = os.path.join(path, label)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face_region = img[y:y + h, x:x + w] 
                        images.append(face_region)
                        labels.append(label)
    return images, labels  # I moved the return outside the loop

In [12]:
def preprocess_images(images):
    preprocessed_images = []
    for image in images:
        img = cv2.equalizeHist(image)
        preprocessed_images.append(img)
    return preprocessed_images

In [13]:
def hog_feature_extraction(images):
    hog_features = []
    for image in images:
        image = cv2.resize(image, (64, 64)) # use 64 for more detail/pixels. 32 only produced 0.5 accuracy. 
        hog_feature = hog(image, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
        hog_features.append(hog_feature)
    return np.array(hog_features)

In [14]:
def load_dataset(train_path, test_path):
    X_train, y_train = load_detect_faces(train_path)
    X_test, y_test = load_detect_faces(test_path)
    
    # Preprocess and extract features - use HoG
    X_train = preprocess_images(X_train)
    X_test = preprocess_images(X_test)
    
    X_train_features = hog_feature_extraction(X_train)
    X_test_features = hog_feature_extraction(X_test)
    
    return X_train_features, y_train, X_test_features, y_test


In [15]:
# Load the JAFFE dataset
jaffe_X_train, jaffe_y_train, jaffe_X_test, jaffe_y_test = load_dataset(jaffe_train_path, jaffe_test_path)

# Load the CK dataset
ck_X_train, ck_y_train, ck_X_test, ck_y_test = load_dataset(ck_train_path, ck_test_path)


In [16]:
# Combine the training data
X_train_combined = np.concatenate((jaffe_X_train, ck_X_train), axis=0)
y_train_combined = np.concatenate((jaffe_y_train, ck_y_train), axis=0)

# Combine the testing data
X_test_combined = np.concatenate((jaffe_X_test, ck_X_test), axis=0)
y_test_combined = np.concatenate((jaffe_y_test, ck_y_test), axis=0)


In [17]:
# Train the model
svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=7, gamma='scale'))
svm.fit(X_train_combined, y_train_combined)

# Test the model
y_pred_combined = svm.predict(X_test_combined)

print("Accuracy:", accuracy_score(y_test_combined, y_pred_combined))
print("Confusion Matrix:\n", confusion_matrix(y_test_combined, y_pred_combined))


Accuracy: 0.6717171717171717
Confusion Matrix:
 [[ 7  0  0  1  1  0  0  0  0  0  0  0]
 [ 0  9  0  0  1  0  0  0  0  0  0  0]
 [ 0  3  6  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  9  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  8  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  5  0  0 12  1  0]
 [ 0  0  0  0  0  0  0  0  0  3  4  0]
 [ 0  0  0  0  0  0  0  0 26  4  0  0]
 [ 0  0  0  0  0  0  0  0  3 27  1  5]
 [ 0  0  0  0  0  0  1  0  0 10  6  0]
 [ 0  0  0  0  0  0  0  0  0 10  1 24]]


In [19]:
# Test on JAFFE data
jaffe_y_pred = svm.predict(jaffe_X_test)
print("JAFFE Accuracy:", accuracy_score(jaffe_y_test, jaffe_y_pred))
print("JAFFE Confusion Matrix:\n", confusion_matrix(jaffe_y_test, jaffe_y_pred))

# Test on CK data
ck_y_pred = svm.predict(ck_X_test)
print("CK Accuracy:", accuracy_score(ck_y_test, ck_y_pred))
print("CK Confusion Matrix:\n", confusion_matrix(ck_y_test, ck_y_pred))


JAFFE Accuracy: 0.8181818181818182
JAFFE Confusion Matrix:
 [[7 0 0 1 1 0]
 [0 9 0 0 1 0]
 [0 3 6 0 0 0]
 [0 0 0 9 0 0]
 [0 1 0 0 8 0]
 [0 3 0 0 0 6]]
CK Accuracy: 0.6153846153846154
CK Confusion Matrix:
 [[ 5  0  0 12  1  0]
 [ 0  0  0  3  4  0]
 [ 0  0 26  4  0  0]
 [ 0  0  3 27  1  5]
 [ 1  0  0 10  6  0]
 [ 0  0  0 10  1 24]]
